In [12]:
import glob
import pandas as pd
leactures_CRN = glob.glob('lectures-CRNs/*.csv')
groupedStudents = glob.glob('groupedStudents/*.csv')
my_CRN='CRNs/roster_CRN_71160.csv'
my_CRN_df=pd.read_csv(my_CRN)
# Concatenate the list of dataframes
groupedStudents_df = pd.concat([pd.read_csv(i) for i in groupedStudents], ignore_index=True)
# Assuming you have two dataframes: dfA and dfB
# Replace 'common_column_A' and 'common_column_B' with the actual column names

# To find missing rows in dfB
missing_students_df = my_CRN_df.merge(groupedStudents_df, 
                             left_on='Student ID', 
                             right_on='SIS Login ID', 

                             how='left', indicator=True).query(
                                 '_merge == "left_only"').drop('_merge', axis=1)

print(missing_students_df.info())


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1 entries, 10 to 10
Columns: 117 entries, # to Quiz 1- Requires Respondus LockDown Browser (148211)
dtypes: float64(44), int64(4), object(69)
memory usage: 944.0+ bytes
None


In [13]:
import pandas as pd

# Path to the CSV file with many records
# main_csv_file = 'main_file.csv'  # Replace with your file path

# List of CSV file paths to check against
# leactures_CRN = ['file1.csv', 'file2.csv', 'file3.csv']  # Add your CSV file paths here

# Read the main CSV file into a dataframe
# main_df = pd.read_csv(main_csv_file)
missing_students_df
# Create an empty dataframe to store the matching records
matching_records_df = pd.DataFrame(columns=missing_students_df.columns)

# Iterate over each record in the main CSV file
for index, row in missing_students_df.iterrows():
    target_id = row['Student ID']  # Assuming 'id' is the common ID column, replace it with the actual column name

    # Iterate through other CSV files and check for the target ID
    for other_csv_file in leactures_CRN:
        try:
            # Read the other CSV file into a dataframe
            other_df = pd.read_csv(other_csv_file, skiprows=1)

            # Check if the target ID exists in the other dataframe
            if target_id in other_df['Student ID'].values:
                # Append the matching record to the new dataframe
                matching_records_df = matching_records_df.append(row, ignore_index=True)
                matching_records_df.loc[matching_records_df["Student ID"]==target_id, "lectures-CRN"]=other_csv_file.replace(".csv","").split('_')[2]
                print(f"Record with ID {target_id} found in {other_csv_file}")
                # Optionally, you can break out of the loop if you only want to find the first occurrence
                break
        except pd.errors.EmptyDataError:
            # Handle empty CSV files if needed
            print(f"Skipping empty file: {other_csv_file}")
        except pd.errors.ParserError:
            # Handle parsing errors if needed
            print(f"Error parsing file: {other_csv_file}")

# Save the matching records to a new CSV file
matching_records_df = matching_records_df.dropna(axis=1, how='all')
matching_records_df.to_csv('ungrouped-Student.csv', index=False)


Record with ID 100855555 found in lectures-CRNs\roster_CRN_75526.csv


C:\Users\Abdelrahman Elewah\AppData\Local\Temp\ipykernel_22612\4025982845.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matching_records_df = matching_records_df.append(row, ignore_index=True)
